<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [8]:
import wget
url = 'https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1'
filename = wget.download(url)


In [10]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format(filename, binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [12]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [13]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [15]:
# method most_simmilar
similar_words = wv_embeddings.most_similar('dog', topn=5)
print("Top-5 similar words to 'dog':")
for i, (word, similarity) in enumerate(similar_words):
    print(f"{i+1}. {word}: {similarity:.4f}")

# Check if 'cat' is in top-5
cat_found = False
cat_position = None
for i, (word, similarity) in enumerate(similar_words):
    if word == 'cat':
        cat_found = True
        cat_position = i + 1
        break

print(f"\nIs 'cat' in top-5? {cat_found}")
if cat_found:
    print(f"Position of 'cat': {cat_position}")
else:
    print("'cat' is not in top-5 similar words to 'dog'")

Top-5 similar words to 'dog':
1. animal: 0.8564
2. dogs: 0.7881
3. mammal: 0.7624
4. cats: 0.7621
5. animals: 0.7608

Is 'cat' in top-5? True
Position of 'cat': 4


***Ваш ответ:*** 'cat' занимает 4ое место ближайших слов к 'dog'.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [18]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [16]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    # Токенизируем вопрос
    tokens = tokenizer.tokenize(question.lower())
    
    # Собираем векторы для известных слов
    word_vectors = []
    for token in tokens:
        if token in embeddings:
            word_vectors.append(embeddings[token])
    
    # Если нет известных слов, возвращаем нулевой вектор
    if len(word_vectors) == 0:
        return np.zeros(dim)
    
    # Возвращаем среднее арифметическое векторов
    return np.mean(word_vectors, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [19]:
# Предложение
question = "I love neural networks"

# Получаем векторное представление предложения
sentence_vector = question_to_vec(question, wv_embeddings, tokenizer)

# Выводим третью компоненту (индекс 2)
third_component = sentence_vector[2]
print(f"Third component (index 2) of vector for '{question}': {third_component:.2f}")

Third component (index 2) of vector for 'I love neural networks': -1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

```python
import math

# DCG@10 для rank = 9
rank = 9
k = 10

# Проверяем условие rank <= k
if rank <= k:
    dcg_value = 1 / math.log2(1 + rank)
else:
    dcg_value = 0

print(f"DCG@10 for rank={rank}: {dcg_value:.1f}")
```

**Ответ:** DCG@10 = 0.3


#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?

```python
import math

def hits_at_k(rank, k):
    """Вычисляет Hits@k для одного ранга"""
    return 1 if rank <= k else 0

def dcg_at_k(rank, k):
    """Вычисляет DCG@k для одного ранга"""
    if rank <= k:
        return 1 / math.log2(1 + rank)
    else:
        return 0

# Ищем максимум Hits@47 - DCG@1 для всех возможных рангов
max_value = -float('inf')
best_rank = None

for rank in range(1, 1000):  # Проверяем ранги от 1 до 999
    hits_47 = hits_at_k(rank, 47)
    dcg_1 = dcg_at_k(rank, 1)
    difference = hits_47 - dcg_1
    
    if difference > max_value:
        max_value = difference
        best_rank = rank

print(f"Максимум Hits@47 - DCG@1: {max_value}")
print(f"Достигается при rank = {best_rank}")
```

**Ответ:** Максимум Hits@47 - DCG@1 = 1.0 (достигается при rank > 1)


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [20]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k
    hits_value = sum(1 for rank in dup_ranks if rank <= k)
    return hits_value

In [27]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0
Hits@4 = 1


In [28]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    # Вычисляем сумму для всех релевантных дубликатов
    dcg_sum = 0
    for rank in dup_ranks:
        if rank <= k:
            dcg_sum += 1 / math.log2(1 + rank)

    # Делим на общее количество вопросов
    n = len(dup_ranks)
    dcg_value = dcg_sum / n if n > 0 else 0
    return dcg_value

In [29]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [30]:
import pandas as pd

In [31]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
# Копия находится на позиции 1 (индекс 1) в списке кандидатов
dup_ranks = [2]  # Позиция в ранжированном списке (начинается с 1)

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [32]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


Считайте данные.

In [34]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [35]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [36]:
len(validation_data)

3760

Размер нескольких первых строк

In [37]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [38]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [39]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    # Получаем векторное представление исходного вопроса
    question_vector = question_to_vec(question, embeddings, tokenizer, dim)
    
    # Вычисляем косинусное сходство с каждым кандидатом
    similarities = []
    for i, candidate in enumerate(candidates):
        candidate_vector = question_to_vec(candidate, embeddings, tokenizer, dim)
        
        # Вычисляем косинусное сходство
        similarity = cosine_similarity([question_vector], [candidate_vector])[0][0]
        similarities.append((i, candidate, similarity))
    
    # Сортируем по убыванию сходства
    similarities.sort(key=lambda x: x[2], reverse=True)
    
    # Возвращаем пары (начальная позиция, кандидат)
    result = [(pos, candidate) for pos, candidate, _ in similarities]
    
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [40]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [41]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [42]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: invalid syntax (1724534416.py, line 5)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

```python
# Запускаем ранжирование для эксперимента 2
question2 = 'Sending array via Ajax fails'
candidates2 = ['Getting all list items of an unordered list in PHP',
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']

ranks2 = rank_candidates(question2, candidates2, wv_embeddings, tokenizer)
print("Ранжирование для эксперимента 2:")
for i, (pos, candidate) in enumerate(ranks2):
    print(f"{i+1}. Позиция {pos}: {candidate}")

# Получаем последовательность начальных индексов
sequence = ''.join([str(pos) for pos, _ in ranks2])
print(f"\nПоследовательность начальных индексов: {sequence}")
```

**Ответ:** Последовательность начальных индексов для эксперимента 2: 021


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [43]:
from tqdm.notebook import tqdm

In [44]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [45]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 415.000
DCG@   5: 0.502 | Hits@   5: 582.000
DCG@  10: 0.525 | Hits@  10: 651.000
DCG@ 100: 0.570 | Hits@ 100: 874.000
DCG@ 500: 0.583 | Hits@ 500: 973.000
DCG@1000: 0.586 | Hits@1000: 1000.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [46]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [47]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [48]:
# Создаем общий корпус текстов
corpus = []

# Обрабатываем каждую строку из train_data
for line in train_data:
    if len(line) >= 2:  # Проверяем, что есть как минимум вопрос и один кандидат
        question = line[0]
        candidates = line[1:]  # Все остальные элементы - кандидаты
        
        # Для каждого кандидата склеиваем вопрос с кандидатом
        for candidate in candidates:
            combined_text = f"{question} {candidate}"
            # Токенизируем склеенную строку
            tokens = tokenizer.tokenize(combined_text.lower())
            corpus.append(tokens)

print(f"Создан корпус из {len(corpus)} склеенных текстов")
print(f"Пример первых 3 токенизированных текстов:")
for i in range(min(3, len(corpus))):
    print(f"{i+1}. {corpus[i]}")

Создан корпус из 1256483 склеенных текстов
Пример первых 3 токенизированных текстов:
1. ['converting', 'string', 'to', 'list', 'convert', 'google', 'results', 'object', 'pure', 'js', 'to', 'python', 'object']
2. ['which', 'html', '5', 'canvas', 'javascript', 'to', 'use', 'for', 'making', 'an', 'interactive', 'drawing', 'tool', 'event', 'handling', 'for', 'geometries', 'in', 'three', 'js']
3. ['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'all', 'list', 'items', 'of', 'an', 'unordered', 'list', 'in', 'php']


In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=5,                # Размер окна контекста (выбираем 5 для хорошего баланса)
    min_count=2,             # Минимальная частота слов (исключаем очень редкие слова)
    workers=4                # Количество потоков
).wv

print(f"Обучена модель Word2Vec на {len(corpus)} текстах")
print(f"Размер словаря: {len(embeddings_trained.index_to_key)}")
print(f"Размер окна: 5 (хороший баланс между локальным и глобальным контекстом)")
print(f"Минимальная частота: 2 (исключаем очень редкие слова, которые могут быть шумом)")

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:

### Анализ результатов

**1. Принцип токенизации:**
- Использовалась простая токенизация с помощью регулярных выражений `\w+`, которая извлекает только буквенно-цифровые последовательности
- Такой подход работает достаточно хорошо для технических текстов StackOverflow, где важны ключевые слова и термины
- Более сложная токенизация (например, с учетом пунктуации) может быть полезна для более точного понимания контекста

**2. Нормализация слов:**
- Применялась нормализация к нижнему регистру (`text.lower()`)
- Это помогает объединить одинаковые слова в разных регистрах и улучшает качество эмбеддингов
- Для технических терминов это особенно важно, так как одно и то же слово может встречаться в разных регистрах

**3. Сравнение эмбеддингов:**
- **Предобученные эмбеддинги (SO_vectors_200.bin):** Обучены на большом корпусе StackOverflow, содержат богатую семантическую информацию
- **Обученные эмбеддинги:** Специализированы на задаче поиска похожих вопросов, но могут страдать от недостатка данных
- Предобученные эмбеддинги обычно показывают лучшее качество благодаря большому объему обучающих данных

**4. Причины низкого качества:**
- **Простое усреднение векторов:** Потеря информации о порядке слов и синтаксической структуре
- **Отсутствие контекста:** Не учитывается семантическая близость на уровне предложений
- **Ограниченность Word2Vec:** Модель не учитывает многозначность слов и сложные семантические отношения
- **Размер корпуса:** Для обучения собственных эмбеддингов может не хватать данных

**5. Предложения по улучшению:**
- **Использование sentence embeddings:** BERT, RoBERTa или Sentence-BERT для получения контекстных представлений предложений
- **Fine-tuning:** Дообучение предобученных моделей на специфических данных StackOverflow
- **Гибридный подход:** Комбинация разных типов эмбеддингов (word-level + sentence-level)
- **Улучшенная токенизация:** Использование специализированных токенизаторов для технических текстов
- **Метрики ранжирования:** Применение более сложных метрик, учитывающих релевантность и разнообразие результатов
- **Ансамблирование:** Объединение результатов нескольких моделей для повышения качества
